In [7]:
import pandas as pd

# Load the data
baseline_eye_tracking = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/sed.csv')
eye_tracking_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/sed_01.csv')
eye_tracking_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/sed_02.csv')
eye_tracking_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/sed_03.csv')

# Load psychometric test data for all three seasons
psychometric_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_01.csv')
psychometric_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_02.csv')
psychometric_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_03.csv')

# Rename columns based on actual column names
columns_mapping = {
    'datetime': 'timestamp',
    'pupil': 'pupil_dilation',
    'leftEyeOpen': 'left_blink',
    'rightEyeOpen': 'right_blink'
}

baseline_eye_tracking.rename(columns=columns_mapping, inplace=True)
eye_tracking_01.rename(columns=columns_mapping, inplace=True)
eye_tracking_02.rename(columns=columns_mapping, inplace=True)
eye_tracking_03.rename(columns=columns_mapping, inplace=True)

# Function to clean eye-tracking data
def clean_eye_tracking_data(eye_tracking_data):
    eye_tracking_data['timestamp'] = pd.to_datetime(eye_tracking_data['timestamp'], errors='coerce').dt.tz_localize(None)
    eye_tracking_data = eye_tracking_data.dropna(subset=['timestamp'])
    return eye_tracking_data

# Clean the data
baseline_eye_tracking = clean_eye_tracking_data(baseline_eye_tracking)
eye_tracking_01 = clean_eye_tracking_data(eye_tracking_01)
eye_tracking_02 = clean_eye_tracking_data(eye_tracking_02)
eye_tracking_03 = clean_eye_tracking_data(eye_tracking_03)

# Clean and prepare the psychometric data
psychometric_01.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_02.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_03.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']

psychometric_01['Question Start Time'] = pd.to_datetime(psychometric_01['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Start Time'] = pd.to_datetime(psychometric_02['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Start Time'] = pd.to_datetime(psychometric_03['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_01['Question Answer Time'] = pd.to_datetime(psychometric_01['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Answer Time'] = pd.to_datetime(psychometric_02['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Answer Time'] = pd.to_datetime(psychometric_03['Question Answer Time'], errors='coerce').dt.tz_localize(None)

psychometric_01 = psychometric_01.dropna(subset=['Question Start Time'])
psychometric_02 = psychometric_02.dropna(subset=['Question Start Time'])
psychometric_03 = psychometric_03.dropna(subset=['Question Start Time'])

# Filter for HADS questions
questions_hads_01 = psychometric_01[psychometric_01['Type'] == 'HADS'].copy()
questions_hads_02 = psychometric_02[psychometric_02['Type'] == 'HADS'].copy()
questions_hads_03 = psychometric_03[psychometric_03['Type'] == 'HADS'].copy()

# Function to filter eye-tracking data based on HADS question times
def filter_eye_tracking_data(eye_tracking_data, question):
    start_time = question['Question Start Time']
    end_time = question['Question Answer Time']
    return eye_tracking_data[(eye_tracking_data['timestamp'] >= start_time) & (eye_tracking_data['timestamp'] <= end_time)]

# Function to calculate eye-tracking metrics
def calculate_eye_tracking_metrics(eye_tracking_data):
    average_pupil_dilation = eye_tracking_data['pupil_dilation'].mean()
    blink_detection_threshold = 1.0  # Threshold value to detect closed eyes
    left_blink_count = ((eye_tracking_data['left_blink'] > blink_detection_threshold) & 
                        (eye_tracking_data['left_blink'].shift(-1) <= blink_detection_threshold)).sum()
    right_blink_count = ((eye_tracking_data['right_blink'] > blink_detection_threshold) & 
                         (eye_tracking_data['right_blink'].shift(-1) <= blink_detection_threshold)).sum()
    total_duration_minutes = (eye_tracking_data['timestamp'].max() - eye_tracking_data['timestamp'].min()).total_seconds() / 60
    left_blink_rate = left_blink_count / total_duration_minutes
    right_blink_rate = right_blink_count / total_duration_minutes
    return average_pupil_dilation, left_blink_rate, right_blink_rate

# Calculate metrics for each HADS question
def calculate_metrics_for_questions(questions, eye_tracking_data):
    results = []
    for _, question in questions.iterrows():
        filtered_data = filter_eye_tracking_data(eye_tracking_data, question)
        if not filtered_data.empty:
            metrics = calculate_eye_tracking_metrics(filtered_data)
            results.append({
                'Question': question['Question'],
                'Start Time': question['Question Start Time'],
                'End Time': question['Question Answer Time'],
                'Score': question['Answer'],
                'Average Pupil Dilation': metrics[0],
                'Average Left Blink Rate': metrics[1],
                'Average Right Blink Rate': metrics[2]
            })
    return pd.DataFrame(results)

# Calculate results for each test
results_01 = calculate_metrics_for_questions(questions_hads_01, eye_tracking_01)
results_02 = calculate_metrics_for_questions(questions_hads_02, eye_tracking_02)
results_03 = calculate_metrics_for_questions(questions_hads_03, eye_tracking_03)

results_01, results_02, results_03

(                                Question              Start Time  \
 0        From time to time; occasionally 2024-06-13 11:30:18.058   
 1                      Not quite so much 2024-06-13 11:30:30.793   
 2                 Yes; but not too badly 2024-06-13 11:30:38.713   
 3                  Not quite so much now 2024-06-13 11:30:46.577   
 4   From time to time; but not too often 2024-06-13 11:30:54.409   
 5                              Sometimes 2024-06-13 11:30:57.497   
 6                                Usually 2024-06-13 11:31:03.497   
 7                             Very often 2024-06-13 11:31:08.945   
 8                           Occasionally 2024-06-13 11:31:14.665   
 9      I may not take quite as much care 2024-06-13 11:31:17.961   
 10                           Quite a lot 2024-06-13 11:31:40.698   
 11            Rather less than I used to 2024-06-13 11:31:50.313   
 12                        Not very often 2024-06-13 11:31:57.265   
 13                             No

In [12]:
import pandas as pd

# Load the data
baseline_eye_tracking = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/sed.csv')
eye_tracking_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/sed_01.csv')
eye_tracking_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/sed_02.csv')
eye_tracking_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/sed_03.csv')

# Load psychometric test data for all three tests
psychometric_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_01.csv')
psychometric_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_02.csv')
psychometric_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_03.csv')

# Rename columns based on actual column names
columns_mapping = {
    'datetime': 'timestamp',
    'pupil': 'pupil_dilation',
    'leftEyeOpen': 'left_blink',
    'rightEyeOpen': 'right_blink'
}

baseline_eye_tracking.rename(columns=columns_mapping, inplace=True)
eye_tracking_01.rename(columns=columns_mapping, inplace=True)
eye_tracking_02.rename(columns=columns_mapping, inplace=True)
eye_tracking_03.rename(columns=columns_mapping, inplace=True)

# Function to clean eye-tracking data
def clean_eye_tracking_data(eye_tracking_data):
    eye_tracking_data['timestamp'] = pd.to_datetime(eye_tracking_data['timestamp'], errors='coerce').dt.tz_localize(None)
    eye_tracking_data = eye_tracking_data.dropna(subset=['timestamp'])
    return eye_tracking_data

# Clean the data
baseline_eye_tracking = clean_eye_tracking_data(baseline_eye_tracking)
eye_tracking_01 = clean_eye_tracking_data(eye_tracking_01)
eye_tracking_02 = clean_eye_tracking_data(eye_tracking_02)
eye_tracking_03 = clean_eye_tracking_data(eye_tracking_03)

# Clean and prepare the psychometric data
psychometric_01.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_02.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_03.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']

psychometric_01['Question Start Time'] = pd.to_datetime(psychometric_01['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Start Time'] = pd.to_datetime(psychometric_02['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Start Time'] = pd.to_datetime(psychometric_03['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_01['Question Answer Time'] = pd.to_datetime(psychometric_01['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Answer Time'] = pd.to_datetime(psychometric_02['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Answer Time'] = pd.to_datetime(psychometric_03['Question Answer Time'], errors='coerce').dt.tz_localize(None)

psychometric_01 = psychometric_01.dropna(subset=['Question Start Time'])
psychometric_02 = psychometric_02.dropna(subset=['Question Start Time'])
psychometric_03 = psychometric_03.dropna(subset=['Question Start Time'])

# Filter for relevant questions
types = ['HADS', 'STAI-S', 'STAI-T', 'BFI', 'FQ']
questions_01 = psychometric_01[psychometric_01['Type'].isin(types)].copy()
questions_02 = psychometric_02[psychometric_02['Type'].isin(types)].copy()
questions_03 = psychometric_03[psychometric_03['Type'].isin(types)].copy()

# Function to filter eye-tracking data based on question times
def filter_eye_tracking_data(eye_tracking_data, question):
    start_time = question['Question Start Time']
    end_time = question['Question Answer Time']
    return eye_tracking_data[(eye_tracking_data['timestamp'] >= start_time) & (eye_tracking_data['timestamp'] <= end_time)]

# Function to calculate eye-tracking metrics
def calculate_eye_tracking_metrics(eye_tracking_data):
    average_pupil_dilation = eye_tracking_data['pupil_dilation'].mean()
    blink_detection_threshold = 1.0  # Threshold value to detect closed eyes
    left_blink_count = ((eye_tracking_data['left_blink'] > blink_detection_threshold) & 
                        (eye_tracking_data['left_blink'].shift(-1) <= blink_detection_threshold)).sum()
    right_blink_count = ((eye_tracking_data['right_blink'] > blink_detection_threshold) & 
                         (eye_tracking_data['right_blink'].shift(-1) <= blink_detection_threshold)).sum()
    total_duration_minutes = (eye_tracking_data['timestamp'].max() - eye_tracking_data['timestamp'].min()).total_seconds() / 60
    left_blink_rate = left_blink_count / total_duration_minutes
    right_blink_rate = right_blink_count / total_duration_minutes
    return average_pupil_dilation, left_blink_rate, right_blink_rate

# Calculate metrics for each question
def calculate_metrics_for_questions(questions, eye_tracking_data):
    results = []
    for _, question in questions.iterrows():
        filtered_data = filter_eye_tracking_data(eye_tracking_data, question)
        if not filtered_data.empty:
            metrics = calculate_eye_tracking_metrics(filtered_data)
            results.append({
                'Type': question['Type'],
                'Question': question['Question'],
                'Start Time': question['Question Start Time'],
                'End Time': question['Question Answer Time'],
                'Score': question['Answer'],
                'Average Pupil Dilation': metrics[0],
                'Average Left Blink Rate': metrics[1],
                'Average Right Blink Rate': metrics[2]
            })
    return pd.DataFrame(results)

# Calculate results for each test
results_01 = calculate_metrics_for_questions(questions_01, eye_tracking_01)
results_02 = calculate_metrics_for_questions(questions_02, eye_tracking_02)
results_03 = calculate_metrics_for_questions(questions_03, eye_tracking_03)

# Combine results into a single dataframe for better presentation
results_01['Test'] = 'Test 01'
results_02['Test'] = 'Test 02'
results_03['Test'] = 'Test 03'

combined_results = pd.concat([results_01, results_02, results_03], ignore_index=True)

# Save the combined results to a new CSV file
combined_results.to_csv('/Users/urmebose/Desktop/People/Nuro/QQ.csv', index=False)

In [18]:
import pandas as pd

# Load the data
baseline_eye_tracking = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/sed.csv')
eye_tracking_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/sed_01.csv')
eye_tracking_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/sed_02.csv')
eye_tracking_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/sed_03.csv')

# Load psychometric test data for all three tests
psychometric_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_01.csv')
psychometric_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_02.csv')
psychometric_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_03.csv')

# Rename columns based on actual column names
columns_mapping = {
    'datetime': 'timestamp',
    'pupil': 'pupil_dilation',
    'leftEyeOpen': 'left_blink',
    'rightEyeOpen': 'right_blink'
}

baseline_eye_tracking.rename(columns=columns_mapping, inplace=True)
eye_tracking_01.rename(columns=columns_mapping, inplace=True)
eye_tracking_02.rename(columns=columns_mapping, inplace=True)
eye_tracking_03.rename(columns=columns_mapping, inplace=True)

# Function to clean eye-tracking data
def clean_eye_tracking_data(eye_tracking_data):
    eye_tracking_data['timestamp'] = pd.to_datetime(eye_tracking_data['timestamp'], errors='coerce').dt.tz_localize(None)
    eye_tracking_data = eye_tracking_data.dropna(subset=['timestamp'])
    return eye_tracking_data

# Clean the data
baseline_eye_tracking = clean_eye_tracking_data(baseline_eye_tracking)
eye_tracking_01 = clean_eye_tracking_data(eye_tracking_01)
eye_tracking_02 = clean_eye_tracking_data(eye_tracking_02)
eye_tracking_03 = clean_eye_tracking_data(eye_tracking_03)

# Clean and prepare the psychometric data
psychometric_01.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_02.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_03.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']

psychometric_01['Question Start Time'] = pd.to_datetime(psychometric_01['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Start Time'] = pd.to_datetime(psychometric_02['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Start Time'] = pd.to_datetime(psychometric_03['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_01['Question Answer Time'] = pd.to_datetime(psychometric_01['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Answer Time'] = pd.to_datetime(psychometric_02['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Answer Time'] = pd.to_datetime(psychometric_03['Question Answer Time'], errors='coerce').dt.tz_localize(None)

psychometric_01 = psychometric_01.dropna(subset=['Question Start Time'])
psychometric_02 = psychometric_02.dropna(subset=['Question Start Time'])
psychometric_03 = psychometric_03.dropna(subset=['Question Start Time'])

# Filter for relevant questions
types = ['HADS', 'STAI-S', 'STAI-T', 'BFI', 'FQ']
questions_01 = psychometric_01[psychometric_01['Type'].isin(types)].copy()
questions_02 = psychometric_02[psychometric_02['Type'].isin(types)].copy()
questions_03 = psychometric_03[psychometric_03['Type'].isin(types)].copy()

# Function to filter eye-tracking data based on question times
def filter_eye_tracking_data(eye_tracking_data, question):
    start_time = question['Question Start Time']
    end_time = question['Question Answer Time']
    return eye_tracking_data[(eye_tracking_data['timestamp'] >= start_time) & (eye_tracking_data['timestamp'] <= end_time)]

# Function to calculate eye-tracking metrics
def calculate_eye_tracking_metrics(eye_tracking_data):
    average_pupil_dilation = eye_tracking_data['pupil_dilation'].mean()
    blink_detection_threshold = 1.0  # Threshold value to detect closed eyes
    left_blink_count = ((eye_tracking_data['left_blink'] > blink_detection_threshold) & 
                        (eye_tracking_data['left_blink'].shift(-1) <= blink_detection_threshold)).sum()
    right_blink_count = ((eye_tracking_data['right_blink'] > blink_detection_threshold) & 
                         (eye_tracking_data['right_blink'].shift(-1) <= blink_detection_threshold)).sum()
    total_duration_minutes = (eye_tracking_data['timestamp'].max() - eye_tracking_data['timestamp'].min()).total_seconds() / 60
    left_blink_rate = left_blink_count / total_duration_minutes
    right_blink_rate = right_blink_count / total_duration_minutes
    return average_pupil_dilation, left_blink_rate, right_blink_rate

# Calculate baseline metrics
baseline_metrics = calculate_eye_tracking_metrics(baseline_eye_tracking)

# Function to detect significant increases compared to baseline
def detect_significant_increase(test_metrics, baseline_metrics):
    pupil_dilation_increase = test_metrics[0] > baseline_metrics[0]
    left_blink_rate_increase = test_metrics[1] > baseline_metrics[1]
    right_blink_rate_increase = test_metrics[2] > baseline_metrics[2]
    return pupil_dilation_increase, left_blink_rate_increase, right_blink_rate_increase

# Calculate metrics for each question and compare to baseline
def calculate_metrics_for_questions(questions, eye_tracking_data, baseline_metrics):
    results = []
    for _, question in questions.iterrows():
        filtered_data = filter_eye_tracking_data(eye_tracking_data, question)
        if not filtered_data.empty:
            metrics = calculate_eye_tracking_metrics(filtered_data)
            significant_increases = detect_significant_increase(metrics, baseline_metrics)
            results.append({
                'Type': question['Type'],
                'Question': question['Question'],
                'Start Time': question['Question Start Time'],
                'End Time': question['Question Answer Time'],
                'Score': question['Answer'],
                'Average Pupil Dilation': metrics[0],
                'Average Left Blink Rate': metrics[1],
                'Average Right Blink Rate': metrics[2],
                'Sign of Anxiety': 'Yes' if any(significant_increases) else 'No'
            })
    return pd.DataFrame(results)

# Calculate results for each test
results_01 = calculate_metrics_for_questions(questions_01, eye_tracking_01, baseline_metrics)
results_02 = calculate_metrics_for_questions(questions_02, eye_tracking_02, baseline_metrics)
results_03 = calculate_metrics_for_questions(questions_03, eye_tracking_03, baseline_metrics)

# Combine results into a single dataframe for better presentation
results_01['Test'] = 'Test 01'
results_02['Test'] = 'Test 02'
results_03['Test'] = 'Test 03'

combined_results = pd.concat([results_01, results_02, results_03], ignore_index=True)

# Save the combined results to a new CSV file
combined_results.to_csv('/Users/urmebose/Desktop/People/Nuro/QQ.csv', index=False)

combined_results.head()  # Display the first few rows of the combined results for verification


,Type,Question,Start Time,End Time,Score,Average Pupil Dilation,Average Left Blink Rate,Average Right Blink Rate,Sign of Anxiety,Test
0,HADS,From time to time; occasionally,2024-06-13 11:30:18.058,2024-06-13 11:30:30.793,1,3.142951,4.722922,4.722922,Yes,Test 01
1,HADS,Not quite so much,2024-06-13 11:30:30.793,2024-06-13 11:30:38.713,1,2.947431,7.589173,0.000000,Yes,Test 01
2,HADS,Yes; but not too badly,2024-06-13 11:30:38.713,2024-06-13 11:30:46.577,2,2.900040,0.000000,7.654038,Yes,Test 01
3,HADS,Not quite so much now,2024-06-13 11:30:46.577,2024-06-13 11:30:54.409,1,2.923094,0.000000,0.000000,Yes,Test 01
4,HADS,From time to time; but not too often,2024-06-13 11:30:54.409,2024-06-13 11:30:57.497,1,2.922475,0.000000,0.000000,Yes,Test 01


In [23]:
# Check and filter the relevant questions properly
# Correcting the total number of questions for each type

import pandas as pd

# Load the data
baseline_eye_tracking = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/sed.csv')
eye_tracking_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/sed_01.csv')
eye_tracking_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/sed_02.csv')
eye_tracking_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/sed_03.csv')

# Load psychometric test data for all three tests
psychometric_01 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_01.csv')
psychometric_02 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_02.csv')
psychometric_03 = pd.read_csv('/Users/urmebose/Desktop/People/Nuro/Psychometric_Test_Results_03.csv')

# Rename columns based on actual column names
columns_mapping = {
    'datetime': 'timestamp',
    'pupil': 'pupil_dilation',
    'leftEyeOpen': 'left_blink',
    'rightEyeOpen': 'right_blink'
}

baseline_eye_tracking.rename(columns=columns_mapping, inplace=True)
eye_tracking_01.rename(columns=columns_mapping, inplace=True)
eye_tracking_02.rename(columns=columns_mapping, inplace=True)
eye_tracking_03.rename(columns=columns_mapping, inplace=True)

# Function to clean eye-tracking data
def clean_eye_tracking_data(eye_tracking_data):
    eye_tracking_data['timestamp'] = pd.to_datetime(eye_tracking_data['timestamp'], errors='coerce').dt.tz_localize(None)
    eye_tracking_data = eye_tracking_data.dropna(subset=['timestamp'])
    return eye_tracking_data

# Clean the data
baseline_eye_tracking = clean_eye_tracking_data(baseline_eye_tracking)
eye_tracking_01 = clean_eye_tracking_data(eye_tracking_01)
eye_tracking_02 = clean_eye_tracking_data(eye_tracking_02)
eye_tracking_03 = clean_eye_tracking_data(eye_tracking_03)

# Clean and prepare the psychometric data
psychometric_01.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_02.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']
psychometric_03.columns = ['Type', 'Test', 'Question', 'Answer', 'Time(s)', 'Question Start Time', 'Question Answer Time']

psychometric_01['Question Start Time'] = pd.to_datetime(psychometric_01['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Start Time'] = pd.to_datetime(psychometric_02['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Start Time'] = pd.to_datetime(psychometric_03['Question Start Time'], errors='coerce').dt.tz_localize(None)
psychometric_01['Question Answer Time'] = pd.to_datetime(psychometric_01['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_02['Question Answer Time'] = pd.to_datetime(psychometric_02['Question Answer Time'], errors='coerce').dt.tz_localize(None)
psychometric_03['Question Answer Time'] = pd.to_datetime(psychometric_03['Question Answer Time'], errors='coerce').dt.tz_localize(None)

psychometric_01 = psychometric_01.dropna(subset=['Question Start Time'])
psychometric_02 = psychometric_02.dropna(subset=['Question Start Time'])
psychometric_03 = psychometric_03.dropna(subset=['Question Start Time'])

# Filter for relevant questions and ensure correct counts
types_count = {
    'HADS': 14,
    'STAI-S': 20,
    'STAI-T': 20,
    'BFI': 10,
    'FQ': 24
}

def filter_correct_questions(df, types_count):
    filtered_df = pd.DataFrame()
    for q_type, count in types_count.items():
        filtered_df = pd.concat([filtered_df, df[df['Type'] == q_type].head(count)])
    return filtered_df

questions_01 = filter_correct_questions(psychometric_01, types_count)
questions_02 = filter_correct_questions(psychometric_02, types_count)
questions_03 = filter_correct_questions(psychometric_03, types_count)

# Function to filter eye-tracking data based on question times
def filter_eye_tracking_data(eye_tracking_data, question):
    start_time = question['Question Start Time']
    end_time = question['Question Answer Time']
    return eye_tracking_data[(eye_tracking_data['timestamp'] >= start_time) & (eye_tracking_data['timestamp'] <= end_time)]

# Function to calculate eye-tracking metrics
def calculate_eye_tracking_metrics(eye_tracking_data):
    average_pupil_dilation = eye_tracking_data['pupil_dilation'].mean()
    blink_detection_threshold = 1.0  # Threshold value to detect closed eyes
    left_blink_count = ((eye_tracking_data['left_blink'] > blink_detection_threshold) & 
                        (eye_tracking_data['left_blink'].shift(-1) <= blink_detection_threshold)).sum()
    right_blink_count = ((eye_tracking_data['right_blink'] > blink_detection_threshold) & 
                         (eye_tracking_data['right_blink'].shift(-1) <= blink_detection_threshold)).sum()
    total_duration_minutes = (eye_tracking_data['timestamp'].max() - eye_tracking_data['timestamp'].min()).total_seconds() / 60
    left_blink_rate = left_blink_count / total_duration_minutes
    right_blink_rate = right_blink_count / total_duration_minutes
    return average_pupil_dilation, left_blink_rate, right_blink_rate

# Calculate baseline metrics
baseline_metrics = calculate_eye_tracking_metrics(baseline_eye_tracking)

# Function to detect significant increases compared to baseline
def detect_significant_increase(test_metrics, baseline_metrics):
    pupil_dilation_increase = test_metrics[0] > baseline_metrics[0]
    left_blink_rate_increase = test_metrics[1] > baseline_metrics[1]
    right_blink_rate_increase = test_metrics[2] > baseline_metrics[2]
    return pupil_dilation_increase, left_blink_rate_increase, right_blink_rate_increase

# Calculate metrics for each question and compare to baseline
def calculate_metrics_for_questions(questions, eye_tracking_data, baseline_metrics):
    results = []
    for _, question in questions.iterrows():
        filtered_data = filter_eye_tracking_data(eye_tracking_data, question)
        if not filtered_data.empty:
            metrics = calculate_eye_tracking_metrics(filtered_data)
            significant_increases = detect_significant_increase(metrics, baseline_metrics)
            results.append({
                'Type': question['Type'],
                'Question': question['Question'],
                'Start Time': question['Question Start Time'],
                'End Time': question['Question Answer Time'],
                'Score': question['Answer'],
                'Average Pupil Dilation': round(metrics[0], 2),
                'Average Left Blink Rate': round(metrics[1], 2),
                'Average Right Blink Rate': round(metrics[2], 2),
                'Sign of Anxiety': 'Yes' if any(significant_increases) else 'No',
                'Pupil Dilation Increase': 'Yes' if significant_increases[0] else 'No',
                'Left Blink Rate Increase': 'Yes' if significant_increases[1] else 'No',
                'Right Blink Rate Increase': 'Yes' if significant_increases[2] else 'No'
            })
    return pd.DataFrame(results)

# Recalculate results for each test with the updated function
results_01 = calculate_metrics_for_questions(questions_01, eye_tracking_01, baseline_metrics)
results_02 = calculate_metrics_for_questions(questions_02, eye_tracking_02, baseline_metrics)
results_03 = calculate_metrics_for_questions(questions_03, eye_tracking_03, baseline_metrics)

# Combine results into a single dataframe for better presentation
results_01['Test'] = 'Test 01'
results_02['Test'] = 'Test 02'
results_03['Test'] = 'Test 03'

combined_results = pd.concat([results_01, results_02, results_03], ignore_index=True)

# Ensure the combined results have the correct number of rows
expected_rows = sum(types_count.values()) * 3
assert len(combined_results) == expected_rows, f"Expected {expected_rows} rows, but got {len(combined_results)}"

# Save the combined results to a new CSV file
combined_results.to_csv('/Users/urmebose/Desktop/People/Nuro/QQ2.csv', index=False)

combined_results.head()  # Display the first few rows of the combined results for verification

,Type,Question,Start Time,End Time,Score,Average Pupil Dilation,Average Left Blink Rate,Average Right Blink Rate,Sign of Anxiety,Pupil Dilation Increase,Left Blink Rate Increase,Right Blink Rate Increase,Test
0,HADS,From time to time; occasionally,2024-06-13 11:30:18.058,2024-06-13 11:30:30.793,1,3.14,4.72,4.72,Yes,Yes,No,No,Test 01
1,HADS,Not quite so much,2024-06-13 11:30:30.793,2024-06-13 11:30:38.713,1,2.95,7.59,0.00,Yes,Yes,No,No,Test 01
2,HADS,Yes; but not too badly,2024-06-13 11:30:38.713,2024-06-13 11:30:46.577,2,2.90,0.00,7.65,Yes,Yes,No,No,Test 01
3,HADS,Not quite so much now,2024-06-13 11:30:46.577,2024-06-13 11:30:54.409,1,2.92,0.00,0.00,Yes,Yes,No,No,Test 01
4,HADS,From time to time; but not too often,2024-06-13 11:30:54.409,2024-06-13 11:30:57.497,1,2.92,0.00,0.00,Yes,Yes,No,No,Test 01
